# Расчет данных по нескольким гео

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Responsum и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from pathlib import Path
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from IPython.display import JSON
from bokeh.models import HoverTool
from bokeh.layouts import gridplot
import logging

from mediascope_api.core import net as msnet
from mediascope_api.responsum import catalogs as rc
from mediascope_api.responsum import tasks as rt

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# pd.set_option("display.max_rows", 200)
# pd.set_option("display.max_colwidth", 50)
# pd.set_option("display.precision", 6)
output_notebook()


# Общие параметры для заданий

Для похожих отчетов часто используются одни и те же параметры, удобно их вынести в отдельные переменные, и затем использовать в коде. Если вдруг понадобится какой-то параметр изменить, его нужно поменять только в одном месте.

###  Расчет аудитории Total Internet

Расчет данных по всем проектам займет достаточно много времени у Responsum, поэтому воспользуемся технической страницей WEB-Index, в которой учитывается вся аудитория.
Техническая страница WEB-Index __site_id = 101__

Важно учитывать в случае расчета статистики OTS:
- если считать с ограничением по Технической странице в медиа-фильтре, то полученная статистика OTS будет показывать общее количество заходов в Интернет за указанный период;
- если считать без ограничений в медиа-фильтре (media_filter = None), то полученная статистика OTS будет показывать общее количество загрузок страниц интернет-проектов, содержащихся в вашем медиа-каталоге, за указанный период;
- статистика OTS корректна для работы с десктопными данными и данными по рекламным кампаниям.

In [ ]:
# задаем параметры
facility = 'mobile' # возможные значения: 'desktop', 'mobile', 'desktop_pre'

# создаем объекты для работы с каталогами и заданиями,
# а так же загружаем каталоги
rcats = rc.ResponsumCats(facility)
rtask = rt.ResponsumTask(facility)

# задаем период расчета
date_from = '2021-03-01'
date_to = '2021-03-31'

# задаем техническую страницу WEB-Index в качестве медиа фильтра
media_filter = 'site = 101'

# задаем типы пользования Интернетом
ut = {'Desktop&Mobile':[1,2,3,4], 'Desktop':[1], 'Mobile':[2,3,4]}

# задаем необходимые гео
geo = {'Большая Москва':'CITY_BM = 1', 
       'Санкт-Петербург':'CITY_BM = 2', 
       '700+ без Мск и Спб':'CITY_TYPE2 = 1 AND CITY_BM != 1 AND CITY_BM != 2', 
       '100-700':'CITY_TYPE2 IN (2, 3)', 
       '100+':'CITY_TYPE2 IN(1, 2, 3)', 
       '0+':'CITY_TYPE2 IN(1,2,3,4)'}

# задаем список статистик для расчета
statistics = ['UnwReach']

# проверяем, что значения параметров установлены верно
rtask.save_report_info(facility, date_from, date_to)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

# Расчет для нескольких гео с разбивкой по Usetype

In [ ]:
%%time

# посчитаем данные в циклах
tasks = []
print('Отправка задач ')

for utype, num in ut.items():
    for demo, logical in geo.items():
        usetypes = num
        project_name = f"{demo} + {utype}"
        demo_filter = f"{logical}"
        structure = {
            "usetype": False,
            "date": "month"
        }

        # формируем из заданных параметров задание для Responsum в формате JSON для каждого из этапов цикла
        task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

        # для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
        tsk = {}
        tsk['project_name'] = project_name
        tsk['usetype'] = f"{utype}"
        tsk['geo'] = f"{demo}"
        tsk['task'] = rtask.send_audience_task(task_json) 
        tasks.append(tsk)
        time.sleep(2)
        print('.', end = '')
print('')

# ждем выполнения
tsks = rtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# получаем результат
results = []
for t in tasks:
    tsk = t['task']
    df_result = rtask.result2table(rtask.get_result(tsk), project_name = t['project_name'])
    df_result['geo'] = t['geo']
    df_result['usetype'] = t['usetype']
    results.append(df_result)
df_result = pd.concat(results)
df_result

## Выгружаем в Excel

In [ ]:
writer = pd.ExcelWriter(rtask.get_excel_filename('UnwReach+Geo_202103'))
df_result.to_excel(writer, 'Report', index=False)
writer.save()